In [1]:
%%capture
!pip install transformers datasets
!python -m pip install huggingface_hub


In [23]:
#@title Select model
# MINE = "jimregan/wav2vec2-swedish-riksdag-200h"
MINE = "jimregan/wav2vec2-swedish-riksdag-100h-transcripts-nolm" #@param ["jimregan/wav2vec2-swedish-riksdag-100h", "jimregan/wav2vec2-swedish-riksdag-200h", "jimregan/wav2vec2-swedish-riksdag-100h-transcripts-nolm"]


In [3]:
from huggingface_hub import notebook_login
notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [24]:
from datasets import load_dataset

cv6 = load_dataset("jimregan/sbtal_riksdag_asr", split="test", use_auth_token=True)

In [25]:
cv6

Dataset({
    features: ['audio', 'text'],
    num_rows: 453
})

In [ ]:
!pip install jiwer

In [ ]:
from datasets import load_metric
wer_metric = load_metric("wer")

In [26]:
from transformers import AutoModelForCTC, Wav2Vec2Processor

mymodel = AutoModelForCTC.from_pretrained(MINE)
myprocessor = Wav2Vec2Processor.from_pretrained(MINE)


In [ ]:
!pip install torchaudio

In [27]:
import torch
import torchaudio
import re

def speech_file_to_array_fn(batch):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“]'
    batch["sentence"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

In [28]:
cv6 = cv6.map(speech_file_to_array_fn)

In [29]:
cv6[0]

{'audio': {'path': 'sweterror-01-1-a_12075_17898.wav',
  'array': array([ 7.62939453e-04, -6.10351562e-05,  1.64794922e-03, ...,
         -3.05175781e-05,  0.00000000e+00, -3.05175781e-05]),
  'sampling_rate': 16000},
 'text': 'i betänkandet tillstyrker utskottet regeringens förslag om anslag för utgiftsområde tre',
 'sentence': 'i betänkandet tillstyrker utskottet regeringens förslag om anslag för utgiftsområde tre'}

In [30]:
import numpy as np

def evaluate(batch):
    audio_dict = batch["audio"][0]["array"]
    inputs = myprocessor(audio_dict, sampling_rate=16_000, return_tensors="pt", padding=True)
    mymodel.to("cuda")
    with torch.no_grad():
        logits = mymodel(inputs.input_values.to("cuda"), attention_mask=inputs.attention_mask.to("cuda")).logits    
    pred_ids = torch.argmax(logits, dim=-1)
    batch["pred_strings"] = myprocessor.batch_decode(pred_ids)
    return batch
result = cv6.map(evaluate, batched=True, batch_size=1)

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

In [31]:
print("WER: {:2f}".format(100 * wer_metric.compute(predictions=result["pred_strings"], references=result["sentence"])))

WER: 14.440600
